In [52]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
from yaml import safe_load, YAMLError
from src.utils import setup_output_folder
from src.snapshots import snapshots_assembly, data_normalization, insert_h5_vector
from src.linear_reduction import SVD
from src.nonlinear_reduction import AutoEncoder

In [129]:
with open("parameters.yaml", "r") as stream:
    try:
        params = safe_load(stream)
    except YAMLError as exc:
        print(exc)

In [130]:
params

{'experiment_name': 'loss_5000',
 'random_state': 42,
 'normalization': {'snapshots': None,
  'svd': 'min_max',
  'autoencoder': None,
  'surrogate': None},
 'splitting': {'strategy': 'temporal',
  'number_of_folds_or_splits': 2,
  'train_size': 0.8,
  'validation_size': 300,
  'test_size': 0.1,
  'gap': 0},
 'snapshots': {'file_type_str': 'h5_fenics',
  'folder': 'data/input',
  'visualization_folder': 'data/visualization',
  'file_name_contains': ['concentration'],
  'dataset': None},
 'svd': {'trunc_basis': 27,
  'normalization': 'min_max',
  'svd_type': 'randomized_svd',
  'power_iterations': 1,
  'oversampling': 20},
 'auto_encoder': {'batch_size': 300,
  'num_epochs': 1,
  'learning_rate': '1e-4',
  'weight_decay': '1e-8',
  'loss_function': 'smooth_l1_loss',
  'loss_parameters': {'beta': 0.2},
  'num_workers': 2,
  'number_of_hidden_layers': 5,
  'hidden_layers_sizes': [256, 128, 64, 32, 16],
  'hidden_layers_activation_function': ['leaky_relu',
   'leaky_relu',
   'leaky_relu',

In [ ]:
filenames, snapshots = snapshots_assembly(params["snapshots"])
snapshots.shape

In [139]:


def pipeline_modes():
    # setup directories
    output_folder = setup_output_folder(params)

    # train_test split
    data_splitter = DataSplitter(params)
    train_data, test_data = data_splitter.preserve_test_data(data)
    #data_splitter.split_data(snapshots)

    # # high dimensional data
    # svd = SVD(snapshots, params, output_folder)
    # svd.fit()
    # svd.plot_singular_values()
    # spatial_modes = svd.u
    # print(f"spatial modes dim: {spatial_modes.shape}")
    # # preprocess high dimensional data
    # normalized_spatial_modes, u_normalization_obj = data_normalization(
    # spatial_modes, params, "svd", transpose=False
    # )    
    # print(f"normalized spatial modes dim: {normalized_spatial_modes.shape}")
    # # fit high dimensional data
    # auto_encoder = AutoEncoder(normalized_spatial_modes, params, output_folder)
    # auto_encoder.fit()
    # auto_encoder.plot_quantities_per_epoch("avg_loss_by_epoch")


In [140]:
pipeline_modes()

AttributeError: 'DataSplitter' object has no attribute 'preserve_test_data'

In [ ]:
svd_step_randomized = SVD(snapshots, params["svd"])
svd_step_randomized.fit()
svd_step_randomized.plot_singular_values()

In [ ]:
projected_snapshots = svd_step_randomized.u.T @ snapshots

projected_snapshots_normalized, u_normalization_obj = data_normalization(
    projected_snapshots, params["svd"], transpose=False
)  # min_max vetores singulares a esquerda
projected_snapshots_normalized.shape

In [ ]:
# u_normalized[0, :].max()

In [ ]:
# vector = u_normalized[:, 0]
# insert_h5_vector(
#     vector,
#     params["snapshots"],
# )

In [ ]:
ae_test_2 = AutoEncoder(projected_snapshots_normalized, params["auto_encoder"])
ae_test_2.fit()

# print(ae_test_2.auto_encoder.encoder)
# print(ae_test_2.auto_encoder.decoder)

In [ ]:
ae_test_2.plot_quantities_per_epoch("avg_loss_by_epoch")

In [ ]:
# 1a analise: erro do autoencoder (apenas)

import torch

f_original_autoencoder = projected_snapshots_normalized[:, 0]
f_reconstructed = ae_test_2.auto_encoder.forward(
    torch.tensor(f_original_autoencoder, dtype=torch.float32)
)
print(f_reconstructed.shape)

import numpy as np

np.linalg.norm(
    f_original_autoencoder - f_reconstructed.detach().numpy()
) / np.linalg.norm(f_original_autoencoder, 2)

In [ ]:
# 2a analise: erro do snapshot
f_reconstructed = ae_test_2.auto_encoder.forward(
    torch.tensor(projected_snapshots_normalized.T, dtype=torch.float32)
)
f_reconstructed_numpy = f_reconstructed.detach().numpy().T
projected_snapshots_return = u_normalization_obj.inverse_transform(
    f_reconstructed_numpy
)
print(projected_snapshots_return.shape)
print(svd_step_randomized.u.shape)
snapshots_returned = svd_step_randomized.u @ projected_snapshots_return

import numpy as np

snapshot_number = 2500

np.linalg.norm(
    snapshots[:, snapshot_number] - snapshots_returned[:, snapshot_number]
) / np.linalg.norm(snapshots[:, snapshot_number], 2)

vector = snapshots_returned[:, snapshot_number]
insert_h5_vector(
    vector,
    params["snapshots"],
)

vector = snapshots[:, snapshot_number]
insert_h5_vector(vector, params["snapshots"], vector_dir="original_vector")

In [ ]:
# TODO: train_test_split
# TODO: organizar plots, geração de erros e dados
# TODO: jogar no google docs
# TODO: surrogate